### Import Libraries

In [287]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Load Dataset

In [288]:
# Load csv
df = pd.read_csv("data/loan.csv")

/tmp/ipykernel_33864/1250389818.py:2: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/loan.csv")


In [289]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Columns: 111 entries, id to total_il_high_credit_limit
dtypes: float64(74), int64(13), object(24)
memory usage: 33.6+ MB


### Data understanding

Going through the data we have found out that 54 columns are only filled with NaN, so we decided to drop those column.
- missing_columns: 'mths_since_last_major_derog, annual_inc_joint, dti_joint, verification_status_joint, tot_coll_amt, tot_cur_bal, open_acc_6m, open_il_6m, open_il_12m, open_il_24m, mths_since_rcnt_il, total_bal_il, il_util, open_rv_12m, open_rv_24m, max_bal_bc, all_util, total_rev_hi_lim, inq_fi, total_cu_tl, inq_last_12m, acc_open_past_24mths, avg_cur_bal, bc_open_to_buy, bc_util, mo_sin_old_il_acct, mo_sin_old_rev_tl_op, mo_sin_rcnt_rev_tl_op, mo_sin_rcnt_tl, mort_acc, mths_since_recent_bc, mths_since_recent_bc_dlq, mths_since_recent_inq, mths_since_recent_revol_delinq, num_accts_ever_120_pd, num_actv_bc_tl, num_actv_rev_tl, num_bc_sats, num_bc_tl, num_il_tl, num_op_rev_tl, num_rev_accts, num_rev_tl_bal_gt_0, num_sats, num_tl_120dpd_2m, num_tl_30dpd, num_tl_90g_dpd_24m, num_tl_op_past_12m, pct_tl_nvr_dlq, percent_bc_gt_75, tot_hi_cred_lim, total_bal_ex_mort, total_bc_limit, total_il_high_credit_limit'
- earliest_cr_line

In [290]:
df.dropna(inplace=True, axis=1, how="all")

In [291]:
column_with_all_same_data = df[df.columns[df.nunique() == 1]].columns

In [292]:
df.drop(columns=column_with_all_same_data, axis = 1, inplace=True)

In [293]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Data columns (total 48 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       39717 non-null  int64  
 1   member_id                39717 non-null  int64  
 2   loan_amnt                39717 non-null  int64  
 3   funded_amnt              39717 non-null  int64  
 4   funded_amnt_inv          39717 non-null  float64
 5   term                     39717 non-null  object 
 6   int_rate                 39717 non-null  object 
 7   installment              39717 non-null  float64
 8   grade                    39717 non-null  object 
 9   sub_grade                39717 non-null  object 
 10  emp_title                37258 non-null  object 
 11  emp_length               38642 non-null  object 
 12  home_ownership           39717 non-null  object 
 13  annual_inc               39717 non-null  float64
 14  verification_status   

Term contains 36 months and 60 months, we are going to convert it in interger with values 36 and 60.

In [294]:
df.term = df.term.apply(lambda x: int(x.replace(" months", "")))

int_rate is a string representation of percentage like 10%, we are going to normalize it and make it float

In [295]:
df.int_rate = df.int_rate.apply(lambda x: float(x.replace("%", "")))

In [297]:
df.isna().sum().sort_values()

id                             0
delinq_2yrs                    0
earliest_cr_line               0
inq_last_6mths                 0
open_acc                       0
pub_rec                        0
revol_bal                      0
total_acc                      0
out_prncp                      0
out_prncp_inv                  0
total_pymnt                    0
total_pymnt_inv                0
total_rec_prncp                0
total_rec_int                  0
total_rec_late_fee             0
recoveries                     0
collection_recovery_fee        0
last_pymnt_amnt                0
addr_state                     0
zip_code                       0
dti                            0
purpose                        0
int_rate                       0
installment                    0
grade                          0
sub_grade                      0
funded_amnt                    0
home_ownership                 0
term                           0
annual_inc                     0
loan_amnt 

In [302]:
df.revol_bal.value_counts()

0        994
298       14
255       14
1         12
682       11
        ... 
21424      1
30747      1
23862      1
20197      1
85607      1
Name: revol_bal, Length: 21711, dtype: int64